### Zuerst werden die benötigten Python Libraries importiert

* `fiona` wird gebraucht um die Geodaten einzulesen (als Geojson)
* `shapely` kovertiet Geojson zu Geometry Objekten 
* `matplotlib` wird als einfaches GIS zum visualisieren der Resultate verwendedt
* `sklearn`enthält den Clustering Algorithmus
* mittels `numpy` können wir Daten effizent als Vektoren/Matritzen verarbeiten

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import fiona
from shapely.geometry.geo import shape
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KDTree
import numpy as np
from ipywidgets import interactive, interact
import ipywidgets as widgets

### Importieren der Daten mittels Fiona

In [ ]:
data = []
with fiona.open('buildings.gpkg') as src:
    for f in src:
        pt = shape(f['geometry'])
        data.append((pt.x, pt.y))
X = np.array(data)
print(X)

 ### Plotten des Datensatzes

In [ ]:
fig = plt.figure()
plt.scatter(X[:, 0], X[:, 1], s=1)

### Finden von optimalen Werten von eps und min_samples

In [ ]:
def plot_nb_dists(nearest_neighbor, metric='euclidean'):
    """ Plots distance sorted by `neared_neighbor`th

    Args:
        X (list of lists): list with data tuples
        nearest_neighbor (int): nr of nearest neighbor to plot
        metric (string): name of scipy metric function to use
    """
    
    tree = KDTree(X, leaf_size=2) 


    if not isinstance(nearest_neighbor, list):
        nearest_neighbor = [nearest_neighbor]

    max_nn = max(nearest_neighbor)

    dist, _ = tree.query(X, k=max_nn + 1)

    plt.figure()

    for nnb in nearest_neighbor:
        col = dist[:, nnb]
        col.sort()
        plt.plot(col, label="{}th nearest neighbor".format(nnb))
 
    #plt.ylim(0, min(250, max(dist[:, max_nn])))
    plt.ylabel("Distance to k nearest neighbor")
    plt.xlabel("Points sorted according to distance of k nearest neighbor")
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
interact(plot_nb_dists,
         nearest_neighbor=widgets.IntSlider(min=1, max=100, step=1, value=1, continuous_update=False));

### DBSCAN Clustering

In [ ]:
def plot_dbscan(eps, min_samples, metric='euclidean'):
   
    db = DBSCAN(eps=eps,
                min_samples=min_samples,
                metric=metric).fit(X)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    
    #print('Estimated number of clusters: %d' % n_clusters_)
    #print('Estimated number of noise points: %d' % n_noise_)
    
    # #############################################################################
    # Plot result
    
    # Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]

    plt.figure()
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]
    
        class_member_mask = (labels == k)
    
        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor=tuple(col), markersize=2)
    
        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor=tuple(col), markersize=2)
    
    plt.title('Estimated number of clusters: %d' % n_clusters_)
    plt.show()


In [ ]:
interact(plot_dbscan,
         eps=widgets.IntSlider(min=1, max=300, step=1, value=50, continuous_update=False),
         min_samples=widgets.IntSlider(min=0, max=50, step=1, value=10, continuous_update=False));